In [1]:
import os
import zipfile
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Path to the training dataset folder
train_path = "/content/train/train"
# Prepare lists for images and labels
images = []
labels = []
# Iterate through training images
for file_name in os.listdir(train_path):
    try:
        # Read the image
        img = cv2.imread(os.path.join(train_path, file_name))
        if img is not None:
            # Resize the image to 64x64 pixels for consistency
            img = cv2.resize(img, (64, 64))
            # Flatten the image and normalize pixel values
            images.append(img.flatten() / 255.0)
            # Assign label based on file name
            if "cat" in file_name.lower():
                labels.append(0)  # Cat = 0
            elif "dog" in file_name.lower():
                labels.append(1)  # Dog = 1
    except Exception as e:
        print(f"Error processing file {file_name}: {e}")
# Convert to NumPy arrays
images = np.array(images)
labels = np.array(labels)

print(f"Loaded {len(images)} images.")
print(f"Loaded {len(labels)} labels.")

# Dimensionality reduction using PCA
pca = PCA(n_components=100)
images_reduced = pca.fit_transform(images)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images_reduced, labels, test_size=0.2, random_state=42)

# Initialize the SVM model
svm_model = SVC(kernel="linear")

# Train the model
svm_model.fit(X_train, y_train)

# Make predictions
y_pred = svm_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Display classification report
print("Classification Report:", classification_report(y_test, y_pred))

# Confusion matrix visualization
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=["Cat", "Dog"], yticklabels=["Cat", "Dog"])
plt.title("Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()